In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .appName('1.2. BigQuery Storage & Spark SQL - Python')\
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest.jar') \
  .getOrCreate()

In [ ]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [ ]:
#read method will return data frame
table = "bigquery-public-data.wikipedia.pageviews_2020"
df_wiki_pageviews = spark.read \
  .format("bigquery") \
  .option("table", table) \
  .option("filter", "datehour >= '2020-03-01' AND datehour < '2020-03-02'") \
  .load()

df_wiki_pageviews.printSchema()

In [ ]:
#with the tempview we can use spark.sql method.
#spark.sql turn dataframe
#create temp table with method df.createOrReplaceTempView
#to use spark.sql interface we have to create
df_wiki_pageviews.createOrReplaceTempView("wiki_pageviews")
df_wiki_en = spark.sql("""
SELECT 
 title, wiki, views
FROM wiki_pageviews
WHERE views > 1000 AND wiki in ('en', 'en.m')
""").cache()

df_wiki_en

In [ ]:
#to use sql interface we need table or temp table
#using spark.sql interface which allow to use write with sql query interface
df_wiki_en.createOrReplaceTempView("wiki_en")
df_wiki_en_totals = spark.sql("""
SELECT 
 title, 
 SUM(views) as total_views
FROM wiki_en
GROUP BY title
ORDER BY total_views DESC
""")

df_wiki_en_totals

In [ ]:
#write data to bigquery. we need: gcs bucket, bigquery dataset, table
# Update to your GCS bucket
gcs_bucket = 'dataproc-bucket-name'
bq_dataset = 'dataset_name'
bq_table = 'wiki_total_pageviews'

df_wiki_en_totals.write \
  .format("bigquery") \
  .option("table","{}.{}".format(bq_dataset, bq_table)) \
  .option("temporaryGcsBucket", gcs_bucket) \
  .mode('overwrite') \
  .save()